## Project 6

In [1]:
import os
import subprocess
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
import scipy
import requests
from imdbpie import Imdb
import nltk
import matplotlib.pyplot as plt
import urllib
from bs4 import BeautifulSoup
import nltk
import collections
import re
import csv
import psycopg2
%matplotlib inline

### Pre-Work: Write a problem statement 

## Part 1: Acquire the Data

#### 1. Connect to the IMDB API

In [2]:
imdb = Imdb()
imdb = Imdb(anonymize=True)
imdb = Imdb(cache=True)

#### 2. Query the top 250 rated movies in the database

In [3]:
movies = pd.DataFrame(imdb.top_250())

#### 3. Only select the top 25 movies and delete the uncessary rows

In [4]:
movies.sort_values(by='rating', axis=0, ascending=False)
movies = movies[:25]
movies.head()

,can_rate,image,num_votes,rating,tconst,title,type,year
0,True,{u'url': u'http://ia.media-imdb.com/images/M/M...,1677486,9.3,tt0111161,The Shawshank Redemption,feature,1994
1,True,{u'url': u'http://ia.media-imdb.com/images/M/M...,1147737,9.2,tt0068646,The Godfather,feature,1972
2,True,{u'url': u'http://ia.media-imdb.com/images/M/M...,784749,9.0,tt0071562,The Godfather: Part II,feature,1974
3,True,{u'url': u'http://ia.media-imdb.com/images/M/M...,1662731,9.0,tt0468569,The Dark Knight,feature,2008
4,True,{u'url': u'http://ia.media-imdb.com/images/M/M...,858473,8.9,tt0108052,Schindler's List,feature,1993


#### 4. Write the Results to a csv

In [5]:
#movies.to_csv('../Assets/Project6/top25.csv')

## Part 2: Wrangle the text data

#### 1. Convert the listing identification numbers (tconst) from the first dataframe to a list

In [6]:
tconst = movies['tconst'].tolist()

#### 2. Scrape the reviews for the top 25 movies

*Hint*: Use a loop to scrape each page at once

In [7]:
# imdbpie has a function where you can get reviews of the 
# titles by specifying the title Id.
Text = []
Id = []
for i in tconst:
    reviews = imdb.get_title_reviews(i,max_results=15)
    for review in reviews:
        Id.append(i)
        Text.append(review.text)

In [8]:
# Makes all the reviews a DataFrame with its respective movie ID
revs = pd.DataFrame(Id)
revs['Text'] = Text
revs.columns=['Id','Text']

#### 5. Tokenize the Output

In [10]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')
tokens = [tokenizer.tokenize(i) for i in Text] #makes each word an individual string
tag = [nltk.pos_tag(i) for i in tokens] #places a tag on each word(verb,adjective,...)

#Filters the adjectives(JJ) into a list
adj = []
for i in (tag):
    for a,b in i:
        if b == "JJ":
            adj.append(a)
            
# counts the most common adjectives and returns the XX most common           
c = collections.Counter
count = c(adj)
common = [a for a,b in count.most_common(15)]
print 'The most common adjectives are:\n {}'.format(common)

from sklearn.feature_extraction.text import CountVectorizer

# Initialize the CountVectorizer object, this accepts the most common 
# words and compares them to the Text. It returns a binary count.  
vectorizer = CountVectorizer(vocabulary=common,binary=True) 

train_data_features = vectorizer.fit_transform(Text)

# Convert the result to an array
train_data_features = train_data_features.toarray()

# Create a DataFrame
train_data = pd.DataFrame(train_data_features, columns=common)


The most common adjective are:
 [u'great', u'first', u'many', u'other', u'good', u'much', u'same', u'such', u'own', u'real', u'true', u'different', u'original', u'little', u'few']


#### 6. Convert to a Dataframe for Easy Viewing

In [23]:
word_data = pd.concat([revs, train_data], axis=1)
word_data = word_data.drop('Text',1)
word_data = word_data.groupby(word_data['Id'], group_keys=False, as_index=False).apply(lambda x: x.iloc[:,:].max())
word_data

,Id,great,first,many,other,good,much,same,such,own,real,true,different,original,little,few
0,tt0038650,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,tt0047478,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1
2,tt0050083,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1
3,tt0060196,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1
4,tt0068646,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1
5,tt0071562,1,1,1,1,1,1,1,1,1,0,1,1,1,1,0
6,tt0073486,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
7,tt0076759,1,1,1,1,1,1,1,1,0,1,0,1,0,1,1
8,tt0080684,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1
9,tt0099685,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1


#### 8. Write the results to a csv

In [24]:
#word_data.to_csv('../Assets/Project6/word_data.csv')

## Part 3: Combine Tables in PostgreSQL

#### 1. Import your two .csv data files into your Postgre Database as two different tables

For ease, we can call these table1 and table2

#### 2. Connect to database and query the joined set

#### 3. Join the two tables 

#### 4. Select the newly joined table and save two copies of the into dataframes

## Part 4: Parsing and Exploratory Data Analysis

#### 1. Rename the column headings

#### 2. Run a description of the data

#### 3. Visualize the Data

## Part 3: Build the Decision Tree

#### 1. What is our target attribute? 

#### 2. Prepare the data and define the training set

#### 2. Train the Model

#### 3. Set up test data and test the model

#### 5. Check the results

#### 6. What is overfitting and how are we at risk? 